In [ ]:
# Import necessary packages
import os
import numpy as np
import pylab as py
import matplotlib.pyplot as plt
from spisea import synthetic, evolution, atmospheres, reddening, ifmr
from spisea.imf import imf, multiplicity
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import csv

# Paths for isochrones and output
iso_dir = 'isochrones/'
output_dir = 'output_diagrams/'

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Estimation variables
star_index = 0      # Set which star in the CSV to analyze
num_top_predictions = 10  # Control how many top predictions to plot

# AKs values from 0 to 20 in increments of 2
extinction_values = np.arange(0, 22, 2)

# Define static isochrone parameters
dist = 4500
evo_model = evolution.Baraffe15()
atm_func = atmospheres.get_merged_atmosphere
red_law = reddening.RedLawHosek18b()
filt_list = ['jwst,F162M', 'jwst,F182M', 'jwst,F200W', 'jwst,F356W', 'jwst,F405N']
filters = ['m_jwst_F162M', 'm_jwst_F182M', 'm_jwst_F200W', 'm_jwst_F356W', 'm_jwst_F405N']
metallicity = 0
level_ages = np.linspace(1, 10, 19) * 1e6  # Define age array
log_age_arr = np.log10(level_ages)

# Load sample magnitudes, skipping the header row
sample_mags = []
with open('../../../s284-no-errors.csv', mode='r') as file:
    csvFile = csv.reader(file)
    next(csvFile)  # Skip header row
    for lines in csvFile:
        sample_mags.append([float(x) for x in lines])

# Chi-square minimization function
def chi_square_reverse_model(iso_grid, sample_mags):
    results = []
    for i, iso in enumerate(iso_grid):
        for star in iso.points:
            chi_square = sum(((sample_mags[k] - star[filters[k]]) ** 2) / star[filters[k]] for k in range(len(sample_mags)))
            results.append([chi_square, star['mass'], 10 ** log_age_arr[i]])
    
    return sorted(results, key=lambda x: x[0])

# Ensure both isochrone and output directories exist before the loop
os.makedirs(iso_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)


# Loop through each extinction value, delete old isochrones, and generate diagrams
for AKs in extinction_values:
    # Clear previous isochrones
    for file in os.listdir(iso_dir):
        os.remove(os.path.join(iso_dir, file))
    
    # Generate isochrone grid
    instances = np.array([
        synthetic.IsochronePhot(log_age, AKs, dist, metallicity=metallicity,
                                evo_model=evo_model, atm_func=atm_func,
                                red_law=red_law, filters=filt_list,
                                iso_dir=iso_dir)
        for log_age in log_age_arr
    ])

    # Run chi-square minimization and get top results
    sorted_results = chi_square_reverse_model(instances, sample_mags[star_index])
    top_results = sorted_results[:num_top_predictions]

    # Create figure with two subplots side-by-side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
    cmap = plt.get_cmap('coolwarm')

    # Color-magnitude diagram (CMD)
    for i, instance in enumerate(instances):
        color = cmap(i / (len(instances) - 1))
        ax1.plot(instance.points[filters[0]] - instance.points[filters[1]], 
                 instance.points[filters[1]], color=color)
    ax1.set_xlabel('F162M - F182M')
    ax1.set_ylabel('F182M')
    ax1.invert_yaxis()
    ax1.grid(True)  # Add grid lines to CMD

    # Magnitude-magnitude diagram (MMD)
    for i, instance in enumerate(instances):
        color = cmap(i / (len(instances) - 1))
        ax2.plot(instance.points[filters[0]], instance.points[filters[1]], color=color)
    ax2.set_xlabel('F162M')
    ax2.set_ylabel('F182M')
    ax2.invert_xaxis()
    ax2.invert_yaxis()
    ax2.grid(True)  # Add grid lines to MMD

    # Create colorbar for both diagrams
    norm = Normalize(vmin=min(level_ages), vmax=max(level_ages))
    sm = ScalarMappable(norm=norm, cmap=cmap)
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=[ax1, ax2])
    cbar.set_label('Age (millions of years)')
    cbar.set_ticks(level_ages)
    cbar.set_ticklabels([f'{age/1e6:.1f}' for age in level_ages])

    # Plot top results on CMD with color gradient and labels
    for j, (chi_square, mass, age) in enumerate(top_results):
        closest_age_idx = np.argmin(abs(level_ages - age))
        iso_instance = instances[closest_age_idx]
        mass_idx = np.argmin(abs(iso_instance.points['mass'] - mass))
        color = cmap(j / (num_top_predictions - 1))
        ax1.plot(iso_instance.points[mass_idx][filters[0]] - iso_instance.points[mass_idx][filters[1]],
                 iso_instance.points[mass_idx][filters[1]], 'o', color=color, label=f"Mass: {mass:.2f}, Age: {age/1e6:.1f} Myr")
        ax2.plot(iso_instance.points[mass_idx][filters[0]],
                 iso_instance.points[mass_idx][filters[1]], 'o', color=color, label=f"Mass: {mass:.2f}, Age: {age/1e6:.1f} Myr")

    # Plot reference star in both diagrams
    ref_star_color = 'gold'  # Optional: distinguish reference star color
    ref_star_marker = '*'  # Optional: distinguish reference star marker

    ax1.plot(sample_mags[star_index][0] - sample_mags[star_index][1], sample_mags[star_index][1], 
         ref_star_marker, markersize=10, color=ref_star_color, label="Reference Star")

    ax2.plot(sample_mags[star_index][0], 
         sample_mags[star_index][1], 
         ref_star_marker, markersize=10, color=ref_star_color, label="Reference Star")

    # Legends and saving
    ax1.legend()
    ax2.legend()
    fig.suptitle(f"AKs = {AKs}")
    plt.savefig(os.path.join(output_dir, f'CMD_MMD_AKs_{AKs}.png'))
    plt.close()

Isochrone generation took 2.550553 s.
Making photometry for isochrone: log(t) = 6.00  AKs = 0.00  dist = 4500
     Starting at:  2024-10-29 03:46:31.627539   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2344 K  m_jwst_F162M = 21.99
Starting filter: jwst,F182M   Elapsed time: 1.75 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2344 K  m_jwst_F182M = 22.18
Starting filter: jwst,F200W   Elapsed time: 3.49 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2344 K  m_jwst_F200W = 22.04
Starting filter: jwst,F356W   Elapsed time: 5.23 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2344 K  m_jwst_F356W = 21.17
Starting filter: jwst,F405N   Elapsed time: 7.07 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2344 K  m_jwst_F405N = 20.84
      Time taken: 8.97 seconds
Isochrone generation took 1.965805 s.
Making photometry for isochrone: log(t) = 6.18  AKs =

      Time taken: 3.62 seconds
Changing to logg=5.00 for T=  1893 logg=3.97
Isochrone generation took 1.291622 s.
Making photometry for isochrone: log(t) = 6.74  AKs = 0.00  dist = 4500
     Starting at:  2024-10-29 03:47:24.276132   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1893 K  m_jwst_F162M = 23.66
Starting filter: jwst,F182M   Elapsed time: 0.69 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1893 K  m_jwst_F182M = 24.05
Starting filter: jwst,F200W   Elapsed time: 1.39 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1893 K  m_jwst_F200W = 23.79
Starting filter: jwst,F356W   Elapsed time: 2.08 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1893 K  m_jwst_F356W = 22.54
Starting filter: jwst,F405N   Elapsed time: 2.81 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1893 K  m_jwst_F405N = 22.18
      Time taken: 3.56 seconds
Changing to log

Starting filter: jwst,F356W   Elapsed time: 4.28 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1602 K  m_jwst_F356W = 23.04
Starting filter: jwst,F405N   Elapsed time: 5.77 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1602 K  m_jwst_F405N = 22.62
      Time taken: 7.36 seconds
Changing to logg=5.00 for T=  1571 logg=4.05
Isochrone generation took 1.818551 s.
Making photometry for isochrone: log(t) = 7.00  AKs = 0.00  dist = 4500
     Starting at:  2024-10-29 03:48:34.783208   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1571 K  m_jwst_F162M = 25.19
Starting filter: jwst,F182M   Elapsed time: 0.69 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1571 K  m_jwst_F182M = 25.03
Starting filter: jwst,F200W   Elapsed time: 1.39 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1571 K  m_jwst_F200W = 24.71
Starting filter: jwst,F356W   Elapsed time: 2.

Starting filter: jwst,F356W   Elapsed time: 2.54 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2011 K  m_jwst_F356W = 23.48
Starting filter: jwst,F405N   Elapsed time: 3.29 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2011 K  m_jwst_F405N = 23.03
      Time taken: 4.05 seconds
Changing to logg=5.00 for T=  1960 logg=3.95
Isochrone generation took 1.306155 s.
Making photometry for isochrone: log(t) = 6.70  AKs = 2.00  dist = 4500
     Starting at:  2024-10-29 03:49:21.386915   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1960 K  m_jwst_F162M = 27.06
Starting filter: jwst,F182M   Elapsed time: 0.72 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1960 K  m_jwst_F182M = 26.65
Starting filter: jwst,F200W   Elapsed time: 1.48 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1960 K  m_jwst_F200W = 25.91
Starting filter: jwst,F356W   Elapsed time: 2.

Starting filter: jwst,F182M   Elapsed time: 0.71 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1631 K  m_jwst_F182M = 27.58
Starting filter: jwst,F200W   Elapsed time: 1.41 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1631 K  m_jwst_F200W = 26.78
Starting filter: jwst,F356W   Elapsed time: 2.14 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1631 K  m_jwst_F356W = 23.98
Starting filter: jwst,F405N   Elapsed time: 2.89 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1631 K  m_jwst_F405N = 23.49
      Time taken: 3.66 seconds
Changing to logg=5.00 for T=  1602 logg=4.04
Isochrone generation took 1.198902 s.
Making photometry for isochrone: log(t) = 6.98  AKs = 2.00  dist = 4500
     Starting at:  2024-10-29 03:50:05.774148   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1602 K  m_jwst_F162M = 28.72
Starting filter: jwst,F182M   Elapsed time: 0.

Starting filter: jwst,F182M   Elapsed time: 0.70 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2052 K  m_jwst_F182M = 29.11
Starting filter: jwst,F200W   Elapsed time: 1.40 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2052 K  m_jwst_F200W = 27.88
Starting filter: jwst,F356W   Elapsed time: 2.10 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2052 K  m_jwst_F356W = 24.47
Starting filter: jwst,F405N   Elapsed time: 2.84 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2052 K  m_jwst_F405N = 23.94
      Time taken: 3.86 seconds
Isochrone generation took 1.983775 s.
Making photometry for isochrone: log(t) = 6.65  AKs = 4.00  dist = 4500
     Starting at:  2024-10-29 03:50:52.227497   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2011 K  m_jwst_F162M = 30.49
Starting filter: jwst,F182M   Elapsed time: 1.26 seconds
Starting synthetic photometry
M = 

Isochrone generation took 1.254991 s.
Making photometry for isochrone: log(t) = 6.93  AKs = 4.00  dist = 4500
     Starting at:  2024-10-29 03:51:35.194954   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1677 K  m_jwst_F162M = 31.84
Starting filter: jwst,F182M   Elapsed time: 0.71 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1677 K  m_jwst_F182M = 30.10
Starting filter: jwst,F200W   Elapsed time: 1.43 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1677 K  m_jwst_F200W = 28.79
Starting filter: jwst,F356W   Elapsed time: 2.14 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1677 K  m_jwst_F356W = 24.93
Starting filter: jwst,F405N   Elapsed time: 2.89 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1677 K  m_jwst_F405N = 24.39
      Time taken: 3.66 seconds
Changing to logg=5.00 for T=  1631 logg=4.04
Isochrone generation took 1.248563 s.
Making p

      Time taken: 3.56 seconds
Isochrone generation took 1.373605 s.
Making photometry for isochrone: log(t) = 6.54  AKs = 6.00  dist = 4500
     Starting at:  2024-10-29 03:52:36.294917   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2099 K  m_jwst_F162M = 33.67
Starting filter: jwst,F182M   Elapsed time: 0.70 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2099 K  m_jwst_F182M = 31.64
Starting filter: jwst,F200W   Elapsed time: 1.39 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2099 K  m_jwst_F200W = 29.87
Starting filter: jwst,F356W   Elapsed time: 2.09 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2099 K  m_jwst_F356W = 25.44
Starting filter: jwst,F405N   Elapsed time: 2.83 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2099 K  m_jwst_F405N = 24.85
      Time taken: 3.62 seconds
Isochrone generation took 1.335116 s.
Making photometry for 

Starting filter: jwst,F405N   Elapsed time: 2.85 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1773 K  m_jwst_F405N = 25.33
      Time taken: 3.62 seconds
Changing to logg=5.00 for T=  1711 logg=4.02
Isochrone generation took 1.271040 s.
Making photometry for isochrone: log(t) = 6.90  AKs = 6.00  dist = 4500
     Starting at:  2024-10-29 03:53:20.884275   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1711 K  m_jwst_F162M = 35.14
Starting filter: jwst,F182M   Elapsed time: 0.71 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1711 K  m_jwst_F182M = 32.64
Starting filter: jwst,F200W   Elapsed time: 1.40 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1711 K  m_jwst_F200W = 30.81
Starting filter: jwst,F356W   Elapsed time: 2.11 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1711 K  m_jwst_F356W = 25.96
Starting filter: jwst,F405N   Elapsed time: 2.

Starting filter: jwst,F356W   Elapsed time: 2.12 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2189 K  m_jwst_F356W = 26.20
Starting filter: jwst,F405N   Elapsed time: 2.85 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2189 K  m_jwst_F405N = 25.57
      Time taken: 3.61 seconds
Isochrone generation took 1.274908 s.
Making photometry for isochrone: log(t) = 6.48  AKs = 8.00  dist = 4500
     Starting at:  2024-10-29 03:54:06.081499   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2143 K  m_jwst_F162M = 36.92
Starting filter: jwst,F182M   Elapsed time: 0.70 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2143 K  m_jwst_F182M = 34.03
Starting filter: jwst,F200W   Elapsed time: 1.46 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2143 K  m_jwst_F200W = 31.75
Starting filter: jwst,F356W   Elapsed time: 2.17 seconds
Starting synthetic photometry
M = 